<a href="https://colab.research.google.com/github/khaulanauman/learning-ml/blob/main/finetune_roberta_small.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
Connecting this google colab notebook to drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Installing Dependencies

In [2]:
!pip install transformers datasets evaluate torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 832.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Importing the dependencies as variables

In [3]:
import torch
import transformers
from transformers import RobertaTokenizerFast, RobertaForQuestionAnswering, Trainer, TrainingArguments
from datasets import Dataset
import pandas as pd
import evaluate
import numpy as np

In [4]:
Uploading the dataset

SyntaxError: invalid syntax (<ipython-input-4-be87d3a7b268>, line 1)

In [5]:
from google.colab import files
uploaded = files.upload()

Saving medquad-small.csv to medquad-small.csv


In [ ]:
Storing the dataset

In [6]:
dataset = Dataset.from_csv("/content/medquad-small.csv")

Generating train split: 0 examples [00:00, ? examples/s]

In [7]:
print(dataset)

Dataset({
    features: ['question', 'answer', 'source', 'focus_area', 'context'],
    num_rows: 499
})


Using pandas to read the dataset and storing in dataframe

In [8]:
csv_path = "/content/medquad-small.csv"


df = pd.read_csv(csv_path)

# Convert Pandas DataFrame to Hugging Face Dataset
dataset = Dataset.from_pandas(df)

# Display dataset sample
print(dataset)
print(dataset[0])  # Display first training sample


Dataset({
    features: ['question', 'answer', 'source', 'focus_area', 'context'],
    num_rows: 499
})
{'question': 'What is (are) Glaucoma ?', 'answer': "Glaucoma is a group of diseases that can damage the eye's optic nerve and result in vision loss and blindness. While glaucoma can strike anyone, the risk is much greater for people over 60. How Glaucoma Develops  There are several different types of glaucoma. Most of these involve the drainage system within the eye. At the front of the eye there is a small space called the anterior chamber. A clear fluid flows through this chamber and bathes and nourishes the nearby tissues. (Watch the video to learn more about glaucoma. To enlarge the video, click the brackets in the lower right-hand corner. To reduce the video, press the Escape (Esc) button on your keyboard.) In glaucoma, for still unknown reasons, the fluid drains too slowly out of the eye. As the fluid builds up, the pressure inside the eye rises. Unless this pressure is control

Storing roberta's tokenizer in a var to be used later

In [9]:
# Use the fast tokenizer to support char_to_token()
tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

Preprocessing the dataset to give to tokenzier

In [10]:
def preprocess_function(examples):
    questions = [str(q) for q in examples["question"]]
    contexts = [str(c) for c in examples["context"]]  # Use the newly added context
    answers = [str(a) for a in examples["answer"]]

    # Tokenize inputs with context
    tokenized_inputs = tokenizer(
        questions,
        contexts,
        truncation=True,
        padding="max_length",
        max_length=512,
        return_offsets_mapping=True
    )

    start_positions = []
    end_positions = []

    for i, answer in enumerate(answers):
        context = contexts[i]
        start_char = context.find(answer)  # Locate answer in context

        if start_char == -1:
            # If answer is not found, set start and end to 0
            start_positions.append(0)
            end_positions.append(0)
            continue

        end_char = start_char + len(answer)

        # Convert character positions to token positions
        offsets = tokenized_inputs["offset_mapping"][i]
        start_token = None
        end_token = None

        for idx, (start, end) in enumerate(offsets):
            if start_token is None and start_char >= start and start_char < end:
                start_token = idx
            if end_char > start and end_char <= end:
                end_token = idx

        if start_token is None:
            start_token = 0
        if end_token is None:
            end_token = len(tokenized_inputs["input_ids"][i]) - 1

        start_positions.append(start_token)
        end_positions.append(end_token)

    tokenized_inputs.pop("offset_mapping", None)
    tokenized_inputs["start_positions"] = start_positions
    tokenized_inputs["end_positions"] = end_positions

    return tokenized_inputs

# Apply function
tokenized_datasets = dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/499 [00:00<?, ? examples/s]

In [11]:
print(dataset[0])

{'question': 'What is (are) Glaucoma ?', 'answer': "Glaucoma is a group of diseases that can damage the eye's optic nerve and result in vision loss and blindness. While glaucoma can strike anyone, the risk is much greater for people over 60. How Glaucoma Develops  There are several different types of glaucoma. Most of these involve the drainage system within the eye. At the front of the eye there is a small space called the anterior chamber. A clear fluid flows through this chamber and bathes and nourishes the nearby tissues. (Watch the video to learn more about glaucoma. To enlarge the video, click the brackets in the lower right-hand corner. To reduce the video, press the Escape (Esc) button on your keyboard.) In glaucoma, for still unknown reasons, the fluid drains too slowly out of the eye. As the fluid builds up, the pressure inside the eye rises. Unless this pressure is controlled, it may cause damage to the optic nerve and other parts of the eye and result in loss of vision. Ope

In [ ]:
Storing base model to fientune it later

SyntaxError: invalid syntax (<ipython-input-81-b73d2363c316>, line 1)

In [12]:
model = RobertaForQuestionAnswering.from_pretrained("roberta-base")

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForQuestionAnswering were not initialized from the model checkpoint at roberta-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Defining our trainingn arguments

In [13]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    weight_decay=0.01,
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Defining our trainer and giving it the dataset, model args and tokenzier

In [14]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    eval_dataset=tokenized_datasets,  # Using same dataset for evaluation in this example
    tokenizer=tokenizer,
)

<ipython-input-14-c6d9ed44fb4e>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [15]:
import wandb
wandb.init(project="roberta-qa-finetuning")

metric = evaluate.load("squad")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    return metric.compute(predictions=predictions, references=labels)

# Evaluate before fine-tuning
baseline_results = trainer.evaluate()
print("Baseline Performance:", baseline_results)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: khaulanauman (khaulanauman-comsats-university-islamabad) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Baseline Performance: {'eval_loss': 6.310024738311768, 'eval_model_preparation_time': 0.003, 'eval_runtime': 13.6653, 'eval_samples_per_second': 36.516, 'eval_steps_per_second': 2.342}


In [16]:
model.save_pretrained("/content/drive/MyDrive/finetune-roberta")
tokenizer.save_pretrained("/content/drive/MyDrive/finetune-roberta")

('/content/drive/MyDrive/finetune-roberta/tokenizer_config.json',
 '/content/drive/MyDrive/finetune-roberta/special_tokens_map.json',
 '/content/drive/MyDrive/finetune-roberta/vocab.json',
 '/content/drive/MyDrive/finetune-roberta/merges.txt',
 '/content/drive/MyDrive/finetune-roberta/added_tokens.json',
 '/content/drive/MyDrive/finetune-roberta/tokenizer.json')

In [17]:
trainer.train()

Epoch,Training Loss,Validation Loss,Model Preparation Time
1,No log,0.845074,0.003000
2,No log,0.576437,0.003000


TrainOutput(global_step=64, training_loss=1.8119865655899048, metrics={'train_runtime': 128.5958, 'train_samples_per_second': 7.761, 'train_steps_per_second': 0.498, 'total_flos': 260774163222528.0, 'train_loss': 1.8119865655899048, 'epoch': 2.0})

In [18]:
model.save_pretrained("/content/drive/MyDrive/finetune-roberta-small")
tokenizer.save_pretrained("/content/drive/MyDrive/finetune-roberta-small")

('/content/drive/MyDrive/finetune-roberta-small/tokenizer_config.json',
 '/content/drive/MyDrive/finetune-roberta-small/special_tokens_map.json',
 '/content/drive/MyDrive/finetune-roberta-small/vocab.json',
 '/content/drive/MyDrive/finetune-roberta-small/merges.txt',
 '/content/drive/MyDrive/finetune-roberta-small/added_tokens.json',
 '/content/drive/MyDrive/finetune-roberta-small/tokenizer.json')

In [20]:
fine_tuned_results = trainer.evaluate()
print("Fine-Tuned Performance:", fine_tuned_results)


Fine-Tuned Performance: {'eval_loss': 0.5764369368553162, 'eval_model_preparation_time': 0.003, 'eval_runtime': 14.4783, 'eval_samples_per_second': 34.465, 'eval_steps_per_second': 2.21, 'epoch': 2.0}


In [21]:
df = pd.DataFrame({
    "Metric": ["Loss", "Exact Match", "F1 Score"],
    "Before Fine-Tuning": [baseline_results["eval_loss"], baseline_results.get("exact_match", 0), baseline_results.get("f1", 0)],
    "After Fine-Tuning": [fine_tuned_results["eval_loss"], fine_tuned_results.get("exact_match", 0), fine_tuned_results.get("f1", 0)]
})


In [22]:
print(df)

        Metric  Before Fine-Tuning  After Fine-Tuning
0         Loss            6.310025           0.576437
1  Exact Match            0.000000           0.000000
2     F1 Score            0.000000           0.000000


In [23]:
from transformers import pipeline

# Load your fine-tuned model
model_path = "/content/drive/MyDrive/finetune-roberta-small"
qa_pipeline = pipeline("question-answering", model=model_path, tokenizer=model_path)


Device set to use cuda:0


In [24]:

context = "Glaucoma is a group of diseases that can damage the eye's optic nerve and result in vision loss and blindness."
question = "What is glaucoma?"

result = qa_pipeline(question=question, context=context)
print(result)


{'score': 0.011379705742001534, 'start': 109, 'end': 110, 'answer': '.'}
